In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/vfnet/vfnet_r2_101_fpn_mdconv_c3-c5_mstrain_2x_coco_empirical_attention.py')

PREFIX = '../../input/data/'

cfg.img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

cfg.albu_train_transforms =  [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.0625,
        scale_limit=0.0,
        rotate_limit=0,
        interpolation=1,
        p=0.5),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0)
        ],
        p=0.1),
    dict(type='JpegCompression', quality_lower=85, quality_upper=95, p=0.2),
    dict(type='ChannelShuffle', p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0)
        ],
        p=0.1),
]
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Albu',
        transforms=cfg.albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Resize', img_scale=[(1333, 480),(1333,960)], keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
#cfg.data.train.pipeline[2]['img_scale'] = (512, 512)
cfg.data.train.pipeline = cfg.train_pipeline

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 2

#cfg.model.train_cfg.rpn_proposal.nms = dict(type='soft_nms', iou_threshold=0.7)
#cfg.model.test_cfg.rpn.nms = dict(type='soft_nms', iou_threshold=0.7)
#cfg.model.test_cfg.rcnn.score_thr=0.1
#cfg.model.test_cfg.rcnn.nms=dict(type='soft_nms', iou_threshold=0.5)




cfg.seed=2020
cfg.gpu_ids = [0]

cfg.work_dir = './work_dirs/vfnet_r2_swa_wsgn'
#cfg.load_from = 'vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth'
#cfg.load_from = 'cascade_rcnn_r101_fpn_dconv_c3-c5_1x_coco_20200203-3b2f0594.pth'
#cfg.work_dir = './work_dirs/baseline_basescale_albumentations_step2_baselr'
cfg.load_from = 'vfnet_r2_101_dcn_ms_2x_51.1.pth'
#cfg.load_from = 'baseline_step1_epoch_12.pth'
cfg.model.bbox_head.num_classes = 11
#cfg.resume_from=cfg.work_dir+'/epoch_10.pth'
#cfg.model.roi_head.mask_head.num_classes = 11
#cfg.optimizer = dict(type='SGD', lr=2e-2, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

cfg.checkpoint_config = dict(max_keep_ckpts=3,interval=1)

cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[16,22])
cfg.runner = dict(type='EpochBasedRunner', max_epochs=24)

# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=500,
#     warmup_ratio=0.001,
#     step=[14, 19])
# cfg.runner = dict(type='EpochBasedRunner', max_epochs=24)

cfg.log_config = dict(  # config to register logger hook
    interval=160,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook')
    ])
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Albu',
        transforms=[
            dict(
                type='ShiftScaleRotate',
                shift_limit=0.0625,
                scale_limit=0.0,
                rotate_limit=0,
                interpolation=1,
                p=0.5),
            dict(
                type='RandomBrightnessContrast',
                brightness_limit=[0.1, 0.3],
                contrast_limit=[0.1, 0.3],
                p=0.2),
            dict(
                type='OneOf',
                transforms=[
                    dict(
                        type='RGBShift',
                        r_shift_limit=10,
                        g_shift_limit=10,
                        b_shift_limit=10,
                

In [3]:
model = build_detector(cfg.model)

2021-05-17 08:47:38,576 - mmdet - INFO - load model from: open-mmlab://res2net101_v1d_26w_4s
2021-05-17 08:47:38,577 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-17 08:47:38,861 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.convs.0.conv_offset.weight, layer2.0.convs.0.conv_offset.bias, layer2.0.convs.1.conv_offset.weight, layer2.0.convs.1.conv_offset.bias, layer2.0.convs.2.conv_offset.weight, layer2.0.convs.2.conv_offset.bias, layer2.1.convs.0.conv_offset.weight, layer2.1.convs.0.conv_offset.bias, layer2.1.convs.1.conv_offset.weight, layer2.1.convs.1.conv_offset.bias, layer2.1.convs.2.conv_offset.weight, layer2.1.convs.2.conv_offset.bias, layer2.2.convs.0.conv_offset.weight, layer2.2.convs.0.conv_offset.bias, layer2.2.convs.1.conv_offset.weight, layer2.2.convs.1.conv_offset.bias, layer2.2.convs.2.conv_offset.weight, layer2.2.convs.2.conv_offset.bia

In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.79s)
creating index...
index created!


In [5]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-17 08:47:47,202 - mmdet - INFO - load checkpoint from vfnet_r2_101_dcn_ms_2x_51.1.pth
2021-05-17 08:47:47,203 - mmdet - INFO - Use load_from_local loader


Done (t=0.88s)
creating index...
index created!


2021-05-17 08:47:47,686 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.vfnet_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([11, 256, 3, 3]).
size mismatch for bbox_head.vfnet_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([11]).
missing keys in source state_dict: backbone.layer3.0.gen_attention_block.appr_bias, backbone.layer3.0.gen_attention_block.geom_bias, backbone.layer3.0.gen_attention_block.gamma, backbone.layer3.0.gen_attention_block.query_conv.weight, backbone.layer3.0.gen_attention_block.key_conv.weight, backbone.layer3.0.gen_attention_block.value_conv.weight, backbone.layer3.0.gen_attention_block.appr_geom_fc_x.weight, backbone.layer3.0.gen_attention_block.appr_geom_fc_y.weight, backbone.layer3.0.gen_attention_block.proj_conv.weight, backbone.layer3.0.gen_attention_block.proj_conv

KeyboardInterrupt: 